In [2]:
import requests
import json
import pandas as pd

In [109]:
# 스캔을 통해 digital link 획득
# url = "http://localhost:8091/epcis/v2/bizLocations/https%3A%2F%2Fid.oliot.org%2F414%2F880123451006/events"

    
query_json = {
  "name": "status",
  "query": {
    "eventType": [
        "ObjectEvent"
    ],
    "EQ_bizStep": [
      "accepting"  
    ]
  }
}

response = requests.post("http://localhost:8091/epcis/v2/queries", json=query_json) 
response

# 여기서 스캔통해 얻은 링크 검색 > 해당 이벤트의 상태시간, 현재위치 가져오기
# 이때 status는 자산 조회 후 모두 최신거 가져오도록 로직 필요함

<Response [201]>

In [13]:
q_response = requests.get("https://localhost:8091/epcis/v2/queries/status/events", verify=False)  # 쿼리로 이벤트 가져오기

def queryResultToDataframe(response) :
    json_object = json.loads(response.text)
    event = json_object['epcisBody']['queryResults']['resultBody']['eventList']

    df = pd.DataFrame()

    for e in event:
        row_data = {
            'eventTime': e['eventTime'],
            'eventTimeZoneOffset': e['eventTimeZoneOffset'],
            'type': e['type'],
            'action': e['action'],
            'bizStep': e['bizStep'],
            'disposition': e.get('disposition'), 
            'epcList': e['epcList'][0],
            'readPoint': e['readPoint']['id'],
            'bizLocation': e['bizLocation']['id'],
            'management:company': e.get('management:company'),
            'sourceList_type': e['sourceList'][0]['type'] if 'sourceList' in e and e['sourceList'] and 'type' in e['sourceList'][0] else None,
            'sourceList_source': e['sourceList'][0]['source'] if 'sourceList' in e and e['sourceList'] and 'source' in e['sourceList'][0] else None,
            'destinationList_type': e['destinationList'][0]['type'] if 'destinationList' in e and e['destinationList'] and 'type' in e['destinationList'][0] else None,
            'destinationList_source': e['destinationList'][0]['destination'] if 'destinationList' in e and e['destinationList'] and 'destination' in e['destinationList'][0] else None,
        }
        df = df.append(row_data, ignore_index=True)

    return df

df = queryResultToDataframe(q_response)
df


c:\ProgramData\Anaconda3\envs\intown\lib\site-packages\urllib3\connectionpool.py:1052: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,eventTime,eventTimeZoneOffset,type,action,bizStep,disposition,epcList,readPoint,bizLocation,management:company,sourceList_type,sourceList_source,destinationList_type,destinationList_source
0,2023-07-28T04:45:08.999Z,+09:00,ObjectEvent,OBSERVE,accepting,None,https://id.oliot.org/8004/8801234561000,https://id.oliot.org/414/880123451006/254/20201,https://id.oliot.org/414/880123451006,None,location,https://id.oliot.org/414/880123451006/254/20201,location,https://id.oliot.org/414/880123451006/254/20203
1,2023-08-02 17:15:51,+09:00,ObjectEvent,OBSERVE,accepting,active,https://id.oliot.org/8004/8801234562222,https://id.oliot.org/414/880123451006/254/20203,https://id.oliot.org/414/880123451006,None,None,None,None,None
2,2023-08-02 17:15:37,+09:00,ObjectEvent,OBSERVE,accepting,available,https://id.oliot.org/8004/8801234561111,https://id.oliot.org/414/880123451006/254/20203,https://id.oliot.org/414/880123451006,None,None,None,None,None
3,2023-08-02 17:15:12,+09:00,ObjectEvent,OBSERVE,accepting,None,https://id.oliot.org/8004/8801234562222,https://id.oliot.org/414/880123451006/254/30301,https://id.oliot.org/414/880123451006,None,location,https://id.oliot.org/414/880123451006/254/30301,location,https://id.oliot.org/414/880123451006/254/20203
4,2023-08-02 17:14:49,+09:00,ObjectEvent,OBSERVE,accepting,None,https://id.oliot.org/8004/8801234561111,https://id.oliot.org/414/880123451006/254/20201,https://id.oliot.org/414/880123451006,None,location,https://id.oliot.org/414/880123451006/254/20201,location,https://id.oliot.org/414/880123451006/254/20203


In [14]:
df['eventTime'] = pd.to_datetime(df['eventTime'])

In [45]:
# pip install pyOpenSSL
import ssl

import urllib3

In [47]:
print(ssl.OPENSSL_VERSION)

OpenSSL 1.1.0h  27 Mar 2018


In [46]:
http = urllib3.PoolManager()
test_epc = http.request('GET', 'https://127.0.0.1:8091/epcis/v2/epcs', verify=False)
test_epc

MaxRetryError: HTTPSConnectionPool(host='127.0.0.1', port=8091): Max retries exceeded with url: /epcis/v2/epcs (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate (_ssl.c:1045)')))

In [39]:
q_epcs = requests.get("https://localhost:8091/epcis/v2/epcs", verify=False)  # 쿼리로 이벤트 가져오기
epcs_json = json.loads(q_epcs.text)
epcs = epcs_json['member']

table = pd.DataFrame()

for epc in epcs:
  df_idf = df[df['epcList'] == epc]

  # 날짜 정렬 후 최신날짜로 조회
  result = df_idf.sort_values('eventTime', ascending=False)
  diposition_info = result[result['disposition'].isnull() != True]
  source_info = result[result['disposition'].isnull() == True]

  # status = diposition_info.reset_index(drop=True).loc[0,"disposition"]
  # status_date = diposition_info.reset_index(drop=True).loc[0,"eventTime"]
  # now_location = diposition_info.reset_index(drop=True).loc[0,"readPoint"]
  # last_location = source_info.reset_index(drop=True).loc[0,"sourceList_source"]
  # finally_move_date = source_info.reset_index(drop=True).loc[0,"eventTime"]

  # row_data = {
  #   "자산": epc,
  #   "상태": status,
  #   "상태변경일": status_date,
  #   "현재위치": now_location,
  #   "지난위치": last_location,
  #   "마지막이동일": finally_move_date
  # }

  # table = table.append(row_data, ignore_index=True)

# diposition_info.reset_index(drop=True).loc[0,"disposition"]

c:\ProgramData\Anaconda3\envs\intown\lib\site-packages\urllib3\connectionpool.py:1052: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


'available'

---

In [ ]:
query_json = {
  "name": "MRO",
  "query": {
    "eventType": [
        "ObjectEvent"
    ],
    "EQ_bizStep": [
      "inspecting",
      "replacing",
      "repairing",
      "destroying"
    ]
  }
}

response = requests.post("http://localhost:8091/epcis/v2/queries", json=query_json) 
response

In [ ]:
df = queryResultToDataframe(q_response)
df['eventTime'] = pd.to_datetime(df['eventTime'])

In [ ]:
q_epcs = requests.get("http://localhost:8091/epcis/v2/epcs")  # 쿼리로 이벤트 가져오기
epcs_json = json.loads(q_epcs.text)
epcs = epcs_json['member']

table = pd.DataFrame()

for epc in epcs:
  df_idf = df[df['epcList'] == epc]

  # 날짜 정렬 후 최신날짜로 조회
  result = df_idf.sort_values('eventTime', ascending=False)

  # 원하는 정보 가져와 보여줄 테이블 만들기
  # status = result[result['disposition'].isnull() != True].reset_index(drop=True).loc[0,"disposition"]
  # status_date = result[result['disposition'].isnull() != True].reset_index(drop=True).loc[0,"eventTime"]
  # now_location = result[result['disposition'].isnull() != True].reset_index(drop=True).loc[0,"readPoint"]

  row_data = {
    "자산": epc,
    "점검일자": mro_date,
    "점검내용": ,
    "유지보수일자": ,
    "유지보수내용": ,
    "현재위치": now_location,
    "관리업체": company,
    "담당자": manager,
    "담당연락처": contact,
    "기타": contents
  }

  table = table.append(row_data, ignore_index=True)

table